In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train_df=pd.read_csv("C:/Users/강민채/Desktop/KUBIG/ML/Team/data/train.csv")
test_df=pd.read_csv("C:/Users/강민채/Desktop/KUBIG/ML/Team/data/test.csv")

# 0. EDA

In [ ]:
train_df.head()

In [34]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
#신용등급 분포 확인
sns.countplot('credit',data=train_df)
plt.show()

#숫자가 작을수록 높은 신용 등급. 가장 낮은 등급에 속하는 사람이 가장 많음.

In [ ]:
#credit 별 성별 분포 확인
sns.countplot('credit',hue='gender',data=train_df)
plt.show()

In [ ]:
#credit별 부동산 소유 여부
sns.countplot('credit',hue='reality',data=train_df)
plt.show()

In [ ]:
#credit별 income_type
sns.countplot('credit',hue='income_type',data=train_df)
plt.show()

In [ ]:
#credit별 edu_type 비교
sns.countplot('credit',hue='edu_type',data=train_df)
plt.show()

In [ ]:
#credit별 수치형 변수들 평균 비교
train_df.groupby('credit').mean()

In [ ]:
#credit별 수치형 변수들 최댓값 비교
train_df.groupby('credit').max()

In [ ]:
# credit 별 income_total 분포
sns.boxplot(x='credit',y='income_total',data=train_df)

In [ ]:
# 신용 등급에 따른 타 변수들의 분포에는 큰 차이가 없는 듯 함.

In [ ]:
train_df['occyp_type'].value_counts()

# 1. 전처리

결측치 처리

In [ ]:
train_df['occyp_type'].isna().sum()

In [3]:
# occyp_type 결측치 처리
# DAYS_EMPLOYED가 0 이거나 양수이면 실제 무직이니까 No job으로 처리.
employed_nojob=train_df[(train_df['DAYS_EMPLOYED']>=0)&(train_df['occyp_type'].isna()==True)]     #8171 중 4438개가 No job
employed_na=train_df[(train_df['DAYS_EMPLOYED']<0)&(train_df['occyp_type'].isna()==True)] 
employed_data=train_df.dropna()

In [4]:
employed_nojob.fillna({'occyp_type':'No job'},inplace=True)

C:\Users\강민채\AppData\Local\Temp\ipykernel_8360\646914387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employed_nojob.fillna({'occyp_type':'No job'},inplace=True)


In [5]:
train_df_2=pd.concat([employed_na,employed_nojob,employed_data])

In [ ]:
train_df_2.info()    # occyp_type 결측치가 3733개로 줄었음.

In [ ]:
train_df_2['occyp_type'].isna().sum()

>> 남은 occyp_type 결측치는 어떻게 처리할지?/대체할 방법...  > 그냥 제거/보간(최빈값)

In [6]:
# 최빈값
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_df_3=pd.DataFrame(imp.fit_transform(train_df_2), columns=train_df_2.columns,index=train_df_2.index)

# laborers로 대체됨 >>> train_df_3

In [ ]:
# 제거
# train_df_2.dropna(inplace=True)   #제거 후 전체 데이터 22724개로 줄어듦.

이상치 처리

In [ ]:
# Outlier 처리
train_df_2.boxplot('family_size')  

#수치형 변수들 각각 box plot 그려보면, 조정 필요한 건 child num, income_total, family_size

In [ ]:
p1 = train_df_2['child_num'].quantile(0.99)
p2 = train_df_2['child_num'].quantile(0.001)
train_df_3 = train_df_2[(train_df_2['child_num'] < p1) & (train_df_2['child_num'] > p2)]

In [ ]:
p3 = train_df_2['income_total'].quantile(0.99)
p4 = train_df_2['income_total'].quantile(0.001)
train_df_3 = train_df_2[(train_df_2['income_total'] < p3) & (train_df_2['income_total'] > p4)]

In [ ]:
p5 = train_df_2['family_size'].quantile(0.99)
p6 = train_df_2['family_size'].quantile(0.001)
train_df_3 = train_df_2[(train_df_2['family_size'] < p5) & (train_df_2['family_size'] > p6)]

In [ ]:
# 이상치 제거 후 17944개로 데이터 줄어듦.

음수값 자료들에서 결측치(양수값) 처리

In [ ]:
# DAYS_BIRTH
train_df_3['DAYS_BIRTH'].describe()     #모두 음수값

In [ ]:
# DAYS_EMPLOYED 
train_df_3['DAYS_EMPLOYED'].describe()    #양수값이 있음. 0으로 바꿔줘야 함.

In [ ]:
train_df_3['DAYS_EMPLOYED'].loc[train_df_3['DAYS_EMPLOYED']>0]=0

In [ ]:
# begin_month
train_df_3['begin_month'].value_counts()     #모두 0 이하.

>세 변수에서 값이 0인 것들 각각 어떻게 처리할지.. / days_employed는 0이면 무직 / 나머지 두 변수는 0이면 결측치.

결측치 처리 했으니, 음수값 변수들 양수로 변환

In [ ]:
train_df_3["DAYS_EMPLOYED"] = np.abs(train_df_3["DAYS_EMPLOYED"])

train_df_3["DAYS_BIRTH"] = np.abs(train_df_3["DAYS_BIRTH"])

train_df_3["begin_month"] = np.abs(train_df_3["begin_month"]).astype(int)

In [ ]:
train_df_3.head()

>파생변수 생성

날짜 관련 변수들 단위 조정(파생변수)

In [ ]:
# 날짜 관련 변수(birth, employed) 단위 조정. 
# 일 데이터 > 월/주 파생 변수 생성

# DAYS_BIRTH 월/주 파생변수 생성 >> 후에 이 파생변수와 다중공선성을 보이는 변수 제거해야 함.
train_df_3["DAYS_BIRTH_month"] = np.floor(train_df_3["DAYS_BIRTH"] / 30) - ((np.floor(train_df_3["DAYS_BIRTH"] / 30) / 12).astype(int) * 12)
train_df_3["DAYS_BIRTH_month"] = train_df_3["DAYS_BIRTH_month"].astype(int)
train_df_3["DAYS_BIRTH_week"] = np.floor(train_df_3["DAYS_BIRTH"] / 7) - ((np.floor(train_df_3["DAYS_BIRTH"] / 7) / 4).astype(int) * 4)
train_df_3["DAYS_BIRTH_week"] = train_df_3["DAYS_BIRTH_week"].astype(int)

In [ ]:
train_df_3.head()

In [ ]:
# 나이 변수 생성
train_df_3["Age"] = train_df_3["DAYS_BIRTH"] // 365

In [ ]:
# DAYS_EMPLOYED의 월/주 변수 생성
train_df_3["DAYS_EMPLOYED_month"] = np.floor(train_df_3["DAYS_EMPLOYED"] / 30) - ((np.floor(train_df_3["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12)
train_df_3["DAYS_EMPLOYED_month"] = train_df_3["DAYS_EMPLOYED_month"].astype(int)
train_df_3["DAYS_EMPLOYED_week"] = np.floor(train_df_3["DAYS_EMPLOYED"] / 7) - ((np.floor(train_df_3["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4)
train_df_3["DAYS_EMPLOYED_week"] = train_df_3["DAYS_EMPLOYED_week"].astype(int)

중복데이터 식별용 파생변수 생성

In [ ]:
# ID 변수 생성
train_df_3['ID'] = \
train_df_3['child_num'].astype(str) + '_' + train_df_3['income_total'].astype(str) + '_' +\
train_df_3['DAYS_BIRTH'].astype(str) + '_' + train_df_3['DAYS_EMPLOYED'].astype(str) + '_' +\
train_df_3['work_phone'].astype(str) + '_' + train_df_3['phone'].astype(str) + '_' +\
train_df_3['email'].astype(str) + '_' + train_df_3['family_size'].astype(str) + '_' +\
train_df_3['gender'].astype(str) + '_' + train_df_3['car'].astype(str) + '_' +\
train_df_3['reality'].astype(str) + '_' + train_df_3['income_type'].astype(str) + '_' +\
train_df_3['edu_type'].astype(str) + '_' + train_df_3['family_type'].astype(str) + '_' +\
train_df_3['house_type'].astype(str) + '_' + train_df_3['occyp_type'].astype(str)

In [ ]:
train_df_3['ID'].value_counts()   #한 사람이 24개 만드는 등 중복 데이터 식별 가능해짐.

불필요 변수 제거

In [ ]:
noneed=['index','FLAG_MOBIL']
train_df_3.drop(noneed,axis=1,inplace=True)

child_num 변수 drop + 자녀유무 변수 생성

In [38]:
train_df_2['kids']=0

In [42]:
train_df_2['kids'] = np.where(train_df_2['child_num'] !=0, 1, 0)

In [46]:
train_df_2.drop(['child_num'],axis=1,inplace=True)

family_size 큰 값들 따로 묶기

In [48]:
# 기준?
train_df_2['family_size'].value_counts()   # 7 이상?

2.0     14106
1.0      5109
3.0      4632
4.0      2260
5.0       291
6.0        44
7.0         9
15.0        3
9.0         2
20.0        1
Name: family_size, dtype: int64

In [49]:
train_df_2['big_family']=0
train_df_2['big_family']=np.where(train_df_2['family_size']>=7, 1, 0)

In [ ]:
# train_df_2.drop(['family_size'],axis=1,inplace=True)

범주형 변수 수치화

In [ ]:
# 범주형 변수 인코딩 - 다중공선성 여부 고려
from sklearn.preprocessing import OrdinalEncoder
cat=train_df_3.dtypes[train_df_3.dtypes == "object"].index.tolist()

In [ ]:
encoder = OrdinalEncoder()
train_df_3[cat] = encoder.fit_transform(train_df_3[cat], train_df_3['credit'])

In [ ]:
train_df_3['ID'] = train_df_3['ID'].astype('int64')

수치형 자료 표준화/정규화

In [ ]:
num=train_df_3.dtypes[train_df_3.dtypes != "object"].index.tolist()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_df_3[num] = scaler.fit_transform(train_df_3[num])

다중공선성 확인

In [ ]:
corr=train_df_3.corr(method='kendall')
plt.figure(figsize=(20,20))
sns.heatmap(corr,annot=True,cmap="Blues")

# 2. 모델링

다중공선성 확인 및 변수 선택/차원 축소

# 3. 성능평가